## DSBD Project 2 -- LendingClub Peer-to-Peer Lending Prediction
### Intermediate Report

#### Yunhan (Claire) Xu, Qiyang (Cathy) Chen

Starting from 2015, the idea of Peer-to-Peer (P2P) lending emerged, which allows individuals (borrowers) to obtain loans directly from other individuals (investors) through platforms such as LendingClub, Prosper, and Upstart. This kind of "social lending" becomes an alternative financing approach, cutting out the financial institution as the middleman. P2P lending enables investors to acquire a higher return on investment compared to a bank deposit. However, credit risk, one of the biggest concerns for investors, emerges when they make informed investment decisions within the P2P lending procedure. In particular, a default due to failing to make the required payment within the agreed date for a borrower would induce a loss for an investor. 

In this project, we plan to develop a model that will predict whether a LendingClub-approved loan will end up being defaulted or not. The collected dataset includes loan grade, indicating the credit risk from the credit report and loan application. We plan to set the loan grade as the baseline and compare it with our model performance. In particular, we will draw a Precision-Recall Curve for the top 1%, 2%, 3%,..., 100% loans with default probability from high to low. We will also draw a Precision-Recall Curve for loan grade from A to G, and compare it with our models' Precision-Recall Curve. 

The domain of our project is as follows:
- Task (T) - Classification task that predicts defualted loan 
- Performance Measure (P) 
    - Precision: among all the actually defaulted loans, how many of them did we successfully identified
    - Recall: among all the loans that we predicted as default loans, how many of them actually defaulted
    - F-1 Score: weighted average for precision and recall
- Experience (E) - LendingClub database of loan records with loan status that we can use to label a loan as defaulted or not
<!-- machine-learned classification models (SVM, logistic regression, decision trees, random forests, XGBoost, etc.) -->

In sum, considering a classification task of identifying borrowers who are likely to make defaulted loans in LendingClub, we will train the LendingClub database of loan records without the pre-specified loan grades to perform the task and utilize precision, recall, and F-1 score as performance metrics to measure how well we complete the task.  

## 1. Data

The original data contains all the LendingClub loan data from 2007 to 2018, and the corresponding 151 features are related to borrowers' credit history and loan characteristics. We performed some basic data cleaning beforehand:
- For the scoping purpose, we only focus on data from the latest four years, i.e., 2015-2018. 
- There are seven loan statuses: Charged Off, Current, Default, Fully Paid, In Grace Period, Late (16-30 days), Late (31-120 days). As we don't know whether a "Current" loan would be defaulted or not, we disregard those loans. We consider Late (16-30 days), Late (31-120 days), Default, and Charged Off as a Defaulted loan and Fully Paid as a desirable loan.

### Columns we discarded
- 100% missing values: `id`, `member_id`, `url`, `next_pymnt_d`
- The same for all rows: `pymnt_plan`, `out_prncp`, `out_prncp_inv`, and `policy_code`
- Updated in past 2 months -- wouldn't have these values at loan origination: `num_tl_120dpd_2m`, `num_tl_30dpd`, `last_fico_range_low`, and
`last_fico_range_high`
- Hardship-related variables, relating to borrowers who were on a hardship plan -- wouldn't know at the point of loan origination whether or not the borrower will have a hardship: `deferral_term`, `hardship_amount`, `hardship_dpd`, `hardship_end_date`, `hardship_flag`, `hardship_last_payment_amount`, `hardship_length`, `hardship_loan_status`, `hardship_payoff_balance_amount`, `hardship_reason`, `hardship_start_date`, `hardship_status`, `hardship_type`, `orig_projected_additional_accrued_interest`, `payment_plan_start_date`
- String variables provided by borrowers. LendingClub categorized them into other variables: `desc`, `title`, and `emp_title`
- Loan settlement/charged off related variables -- wouldn't know them at the time of funding: `debt_settlement_flag`, `debt_settlement_flag_date`, `settlement_amount`, `settlement_date`, `settlement_percentage`, `settlement_status`, `settlement_term`, `recoveries`, `collection_recovery_fee`
- Loan payment related variables -- wouldn't know them at the time of funding: `funded_amnt`, `funded_amnt_inv`, `last_pymnt_amnt`, `out_prncp`, `out_prncp_inv`, `total_pymnt`, `total_pymnt_inv`, `total_rec_int`, `total_rec_late_fee`, `total_rec_prncp`, `last_pymnt_d`, `last_credit_pull_d`, `disbursement_method`
- `initial_list_status`: as the values "W' and "F" are randomly assigned
- Columns relating to co-borrowers. These columns only have non-null values from issue dates of 2017-03-01 and later (or 2015-10-01 and later for 'dti_joint', 'annual_inc_joint', and 'verification_status_joint'), and have around 97% missing values, so we remove these columns: `annual_inc_joint`, `dti_joint`, `revol_bal_joint`, `sec_app_chargeoff_within_12_mths`, `sec_app_collections_12_mths_ex_med`, `sec_app_earliest_cr_line`, `sec_app_inq_last_6mths`, `sec_app_mort_acc`, `sec_app_mths_since_last_major_derog`, `sec_app_num_rev_accts`, `sec_app_open_acc`, `sec_app_open_act_il`, `sec_app_revol_util`, `verification_status_joint`, `sec_app_fico_range_low`, `sec_app_fico_range_high`
- Other: `zip_code`

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
sns.set(font_scale=1.5)
sns.set_style('ticks')

In [4]:
data = pd.read_csv('LC_15_18.csv').iloc[:, 1:]

## 2. Data Cleaning

In [5]:
# drop additional colmns
data = data.drop(['last_fico_range_low', 'last_fico_range_high'], axis = 1)

In [6]:
# check number of rows and cols in the dataset
data.shape

(927931, 71)

In [7]:
# change data type
data['issue_d'] = pd.to_datetime(data['issue_d'])
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])

In [8]:
# check missing values in the dataset
def missing_values_table(data):
    '''
    generate a table that contains attributes and its corresponing number of missing values 
    and percentage of missing value
    '''
    #Number of null values by column
    mv_df = pd.DataFrame(data.isna().sum(),columns=['Count_Missing'])
    
    #Portion of null values by column
    mv_df['Pctg_Missing'] = mv_df['Count_Missing']/len(data)

    #Sort by Missing_Count
    mv_df = mv_df.sort_values('Count_Missing',ascending=False)  
    
    return mv_df

mv_df = missing_values_table(data)

mv_df.head()

,Count_Missing,Pctg_Missing
mths_since_last_record,753505,0.812027
mths_since_recent_bc_dlq,696123,0.750188
mths_since_last_major_derog,659938,0.711193
mths_since_recent_revol_delinq,600515,0.647155
mths_since_last_delinq,451425,0.486486


In [9]:
# Re-categorize emp_length into 4 categories
def process_emp_len(d):
    '''
    re_categorize the employment length in years to reduce the categories
    '''
    df = d.copy()
    # 0-1 years include null, < 1 year and 1 year
    df['emp_length'] = df['emp_length'].fillna('0-1 years')
    df['emp_length'] = np.where(df['emp_length'].isin([np.nan,None,'< 1 year','1 year']),'0-1 years',df['emp_length'])

    # 2-5 years include 2 years, 3 years, 4 years and 5 years
    df['emp_length'] = np.where(df['emp_length'].isin(['2 years','3 years','4 years', '5 years']),'2-5 years',df['emp_length'])

    # 6-9 years include 6 years, 7 years, 8 years and 9 years
    df['emp_length'] = np.where(df['emp_length'].isin(['6 years','7 years','8 years', '9 years']),'6-9 years',df['emp_length'])

    return df


# For missing values in columns that measure "months since": fill the nulls with maximum observed value +1
def process_month_since_cols(d):
    '''
    fill the null values with the maximum observed value + 1 in 'month since' related columns, 
    the borrowers who have never had delinquencies will have the largest value for number of months 
    since delinquency.
    '''
    df = d.copy()

    mon_since_cols = ['mo_sin_old_il_acct','mths_since_last_delinq','mths_since_last_major_derog',
                     'mths_since_last_record','mths_since_recent_bc_dlq','mths_since_recent_inq',
                     'mths_since_recent_revol_delinq','mo_sin_old_rev_tl_op','mths_since_recent_bc',
                     'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl']

    for i in mon_since_cols:
        df[i].fillna(df[i].max()+1, inplace=True)
    
    return df


# Change grade and sub_grade to numerical rankings (the lower the number, the less risky the loan)
def process_loan_grades(d):
    '''
    turn loan grade and sub_grade into ordinal mapping
    '''
    df = d.copy()

    # turn grade into ordinal mapping
    sorted_grades = sorted(d.grade.unique())
    grade_dict = dict(zip(sorted_grades, range(len(sorted_grades))))
    df['grade'] = df['grade'].map(grade_dict)

    # turn sub_grade into ordinal mapping
    sorted_subgrades = sorted(d.sub_grade.unique())
    subgrade_dict = dict(zip(sorted_subgrades, range(len(sorted_subgrades))))
    df['sub_grade'] = df['sub_grade'].map(subgrade_dict)

    return df


# Process the above transformation and a few more transformations
def process_loan_cols(d):
    df_processed = process_emp_len(d)
    df_processed = process_month_since_cols(df_processed)
    df_processed = process_loan_grades(df_processed)
    
    # add `credit_line_length` by compute the difference between the month the borrower's earliest reported credit line was opened
    # and the month which the loan was funded. Then discard the earliest_cr_line column.
    df_processed['credit_line_length'] = df_processed['issue_d'] - df_processed['earliest_cr_line']
    df_processed = df_processed.drop(['earliest_cr_line'], axis=1)

    # process `revol_util`, `int_rate`, `credit_line_length`
    df_processed['revol_util'] = df_processed['revol_util'].apply(lambda x: x/100)
    df_processed['int_rate'] = df_processed['int_rate'].apply(lambda x: x/100)
    df_processed['credit_line_length'] = df_processed['credit_line_length'].apply(lambda x: x.days)
    
    # generate `default` (binary variable, 0 means not defaulted, 1 means defaulted) and drop `loan_status`
    df_processed['default'] = df_processed['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1, 'Default': 1,
                                                                  'In Grace Period': 1, 'Late (16-30 days)': 1, 
                                                                  'Late (31-120 days)': 1})
    df_processed = df_processed.drop(columns='loan_status')
    
    return df_processed


In [10]:
# create a new pre-process dataset
data_new = process_loan_cols(data)

In [11]:
# check missing values in new dataset
mv_df = missing_values_table(data_new) # the greatest missing percentage of a feature is around 0.01

# get the list of variables that have missing values
mv_list = mv_df[mv_df['Count_Missing'] > 0].index.tolist() # all the variables that have missing values are float

# impute these missing values with its median
for var in mv_list:
    data_new[var] = data_new[var].fillna((data_new[var].median()))

In [12]:
# check the data type for each attribute
types_data = pd.DataFrame(data_new.dtypes, columns=['Types'])

# check the categorical data
cat_var = types_data[types_data['Types'] == 'object'].index.tolist()
cat_var

['term',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'application_type',
 'issue_month']

In [13]:
# create dummy variables for categorical variables
data_new = pd.get_dummies(data_new, columns=cat_var, drop_first=True) 

## 3. Exploratory Data Analysis

In [14]:
data_new.groupby('default')['default'].count()/len(data_new)

default
0    0.756728
1    0.243272
Name: default, dtype: float64

75.67% of the loans are fully paid while 24.32% of the loans are defaulted.

### Feature Importance

Since there are more than 100 variables, we would like to first get a sense of which features are the most predictive by training a simple Random Forest Classifier and looking at the feature importance plot.

In [15]:
# When training the model, we need to first discard some columns:
# 'issue_d' has the date data type, which cannot be used in the rf model
# 'grade', 'sub_grade', and 'int_rate' are determined by loans' and borrowers' features and are directly associated with default risk
rf_data = data_new.drop(columns= ['issue_d', 'grade', 'sub_grade', 'int_rate'])
data_train, data_val = train_test_split(rf_data, test_size=.1, stratify=rf_data['default'], random_state=99)

X_train = data_train.drop(columns='default')
y_train = data_train['default']

X_val = data_val.drop(columns='default')
y_val = data_val['default']

In [16]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=None).fit(X_train, y_train)

importances = pd.DataFrame({'Columns':X_train.columns,'Feature_Importances':rf_model.feature_importances_})
importances = importances.sort_values(by='Feature_Importances',ascending=False)

In [17]:
importances.head(15)

,Columns,Feature_Importances
3,dti,0.031392
1,installment,0.030646
58,term_ 60 months,0.027436
23,bc_open_to_buy,0.025764
52,tot_hi_cred_lim,0.025185
28,mo_sin_old_rev_tl_op,0.025047
0,loan_amnt,0.025037
2,annual_inc,0.024483
22,avg_cur_bal,0.024381
20,total_rev_hi_lim,0.024138


The 15 most important predictors are:
- `dti`: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
- `installment`: The monthly payment owed by the borrower if the loan originates.
- `term`: The number of payments on the loan. Values are in months and can be either 36 or 60.
- `bc_open_to_buy`: Total open to buy on revolving bankcards.
- `loan_amnt`
- `mo_sin_old_rev_tl_op`: Months since oldest revolving account opened.
- `tot_hi_cred_lim`: Total high credit/credit limit.
- `avg_cur_bal`: Average current balance of all accounts.
- `annual_inc`: The self-reported annual income provided by the borrower during registration.
- `total_rev_hi_lim`: Total revolving high credit/credit limit.
- `credit_line_length`: The difference between the month the borrower's earliest reported credit line was opened and the month which the loan was funded.
- `total_bc_limit`: Total bankcard high credit/credit limit.
- `bc_util`: Ratio of total current balance to high credit/credit limit for all bankcard accounts.
- `mo_sin_old_il_acct`: Months since oldest bank installment account opened
- `revol_util`: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

Let's do some EDA on these 15 predictors.

In [18]:
# fig, ax = plt.subplots(figsize=(5,8))
# ax = sns.barplot(x=importances['Feature_Importances'].head(15), y=importances['Columns'].head(15))
# sns.despine()

In [19]:
# change the 'term_ 60 months' variable to float to compute mean
data_new['term_ 60 months'] = data_new['term_ 60 months'].astype(np.float64)

# Predictors related to loans
data_loan = data_new.groupby('default').agg({
                'term_ 60 months': 'mean',
                'installment': 'median',
                'loan_amnt': 'median'
})

data_loan.reset_index(inplace = True)
data_loan

,default,term_ 60 months,installment,loan_amnt
0,0,0.194299,364.70,12000.0
1,1,0.382887,408.22,14725.0


`term` is the number of payments on the loan. Values are in months and can be either 36 or 60. 38.29% of the defaulted loans are 60-months while only 19.43% of the fully paid loans are 60-months. 

Defaulted loans have a higher monthly payment owed by the borrower if the loan originates and a higher loan amount.

In [20]:
# change the 'term_ 60 months' variable back to uint8 data type
data_new['term_ 60 months'] = data_new['term_ 60 months'].astype(np.uint8)

In [21]:
# alt.Chart(data_loan, title = 'The mean of ...').mark_bar().encode(
#     alt.X('default:N', title = 'If default'),
#     alt.Y('term_ 60 months:Q', title = 'If term is 60 month'),
# ).properties(width = 300, height = 300)

In [22]:
# alt.Chart(data_loan, title = 'Defaulted loans have a higher median monthly payment owed by the borrower').mark_bar().encode(
#     alt.X('default:N', title = 'If default'),
#     alt.Y('installment:Q', title = 'Median installment'),
# ).properties(width = 300, height = 300)

In [23]:
# alt.Chart(data_loan, title = 'Defaulted loans have a higher median loan amount').mark_bar().encode(
#     alt.X('default:N', title = 'If default'),
#     alt.Y('loan_amnt:Q', title = 'Median loan amount'),
# ).properties(width = 300, height = 300)

In [26]:
# Predictors related to borrowers' financials
data_new.groupby('default').agg({
    'dti': 'median',
    'bc_open_to_buy': 'median',
    'mo_sin_old_rev_tl_op': 'median',
    'tot_hi_cred_lim': 'median',
    'avg_cur_bal': 'median',
    'annual_inc': 'median',
    'total_rev_hi_lim': 'median',
    'credit_line_length': 'median',
    'total_bc_limit': 'median',
    'bc_util': 'median',
    'mo_sin_old_il_acct': 'median',
    'revol_util': 'median'
})

,dti,bc_open_to_buy,mo_sin_old_rev_tl_op,tot_hi_cred_lim,avg_cur_bal,annual_inc,total_rev_hi_lim,credit_line_length,total_bc_limit,bc_util,mo_sin_old_il_acct,revol_util
default,,,,,,,,,,,,
0,17.50,5636.0,167.0,123653.0,8037.0,67000.0,25500.0,5508.0,16200.0,58.5,131.0,0.483
1,20.07,4177.5,153.0,89510.0,5769.0,62000.0,22600.0,5233.0,13600.0,64.0,128.0,0.529


Borrowers of **defaulted** loans have:
- higher `dti`: debt to income ratio
- lower `bc_open_to_buy`: total open to buy on revolving bankcards
- less `mo_sin_old_rev_tl_op`: months since oldest revolving account opened
- lower `tot_hi_cred_lim`: total high credit/credit limit
- lower `avg_cur_bal`: average current balance of all accounts
- lower `annual_inc`: annual income
- lower `total_rev_hi_lim`: total revolving high credit/credit limit
- less `credit_line_length`: months between the borrower's earliest reported credit line was opened and the loan was funded
- lower `total_bc_limit`: total bankcard high credit/credit limit
- lower `bc_util`: ratil of total current balance to high credit/credit limit for all bankcard accounts
- less `mo_sin_old_il_acct`: months since oldest bank installment account opened
- lower `revol_util`: revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit

than borrowers of **fully paid** loans.

## 4. Models

### Prepare train and test data

We have data from 2015 to 2018. We may use data from 2015 to 2017 as train data and data from 2018 as test data. This could be a reasonable method of splitting because we want to use past data to train the model and predict the existing loans. Let's first take a look at the size of the data each year.

In [33]:
data_new.groupby('issue_year').size()/len(data_new)

issue_year
2015.0    0.407138
2016.0    0.323673
2017.0    0.195842
2018.0    0.073347
dtype: float64

As we typically use 20% of the data as test data, we select data from the second half of 2017 and 2018 as test data. 

In [38]:
# Prepare train data
train_data = data_new[data_new.issue_d < "2017-07-01"]

# Prepare test data
test_data = data_new[data_new.issue_d >= "2017-07-01"]

In [46]:
# Check the size of train and test data
print('The size of train data is:', train_data.shape[0], ', which is', round(train_data.shape[0]/data_new.shape[0], 2), 'of the whole data set', '\n'
'The size of the test data is:', test_data.shape[0], ', which is', round(test_data.shape[0]/data_new.shape[0], 2), 'of the whole data set')

The size of train data is: 774599 , which is 0.83 of the whole data set 
The size of the test data is: 153332 , which is 0.17 of the whole data set


### 4-1. Logistic Regression (Cathy)

### 4-2 Decision Trees